In [23]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup

base_url = r'/home/amit/DataScienceProject/PredictDroughts/'

In [24]:
class Droughts:
    states = None
    url = None
    iterState = None

    def __init__(self, url):
        self.url = url
        self.setUp(self.url)

    def __del__(self):
        self.driver.quit()

    def setUp(self, url):
        self.driver = webdriver.Firefox(executable_path=base_url + "geckodriver")
        self.driver.get(self.url)
        self.driver.maximize_window()
        self.driver.implicitly_wait(10)
        time.sleep(1)
        self.driver.find_element_by_id("atype_chosen").click()
        self.driver.find_element_by_xpath("//li[@class='active-result' and text()='State']").click()
        self.driver.find_element_by_name("datatabl_length").click()
        self.driver.find_element_by_xpath("//select[@name='datatabl_length']/option[text()='All']").click()
        self.driver.find_element_by_xpath("//th[text()='Week']").click()
        selectElement = self.driver.find_element_by_id("asel")
        self.states = [elem.get_attribute("textContent") for elem in selectElement.find_elements_by_tag_name("option")]
    
    def changeState(self):
        dropDownSelect = self.driver.find_element_by_id("asel_chosen")
        dropDownSelect.click()
        dropDownUl = dropDownSelect.find_element_by_class_name("chosen-results")
        dropDownLis = dropDownUl.find_elements_by_class_name('active-result')
        dropDownLis[self.iterState].click()

    def getData(self):
        self.iterState = 0
        data = []
        fipsDf = pd.read_csv(base_url + r'files/fips.csv')
        for state in self.states:
            self.changeState()
            data.append(self.getDataForState(fipsDf))
            self.iterState += 1
        return data

    def getDataForState(self, fipsDf):
        html = self.driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        div = soup.select_one("#datatabl")
        table = pd.read_html(str(div))[0]
        table.insert(0, 'State', self.states[self.iterState], True)
        table.insert(1, 'Postal code', fipsDf['postal code'][self.iterState], True)
        table.insert(2, 'Fips', fipsDf['fips'][self.iterState], True)
        return table

In [25]:
droughts = Droughts(r'https://droughtmonitor.unl.edu/DmData/DataTables.aspx')
droughts

In [26]:
dataframe = pd.DataFrame()
dfList = droughts.getData()
for df in dfList:
    dataframe = dataframe.append(df)
dataframe.reset_index(drop=True, inplace=True)
dataframe

,State,Postal code,Fips,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI
0,Alabama,AL,1,2000-01-04,0.00,100.00,47.98,16.35,0.00,0.0,164
1,Alabama,AL,1,2000-01-11,0.00,100.00,76.88,4.24,0.00,0.0,181
2,Alabama,AL,1,2000-01-18,0.00,100.00,75.71,4.24,0.00,0.0,180
3,Alabama,AL,1,2000-01-25,0.00,100.00,75.67,4.79,0.00,0.0,180
4,Alabama,AL,1,2000-02-01,0.00,100.00,50.07,0.17,0.00,0.0,150
...,...,...,...,...,...,...,...,...,...,...,...
58235,Wyoming,WY,56,2021-05-18,10.89,89.11,70.75,27.15,2.01,0.0,189
58236,Wyoming,WY,56,2021-05-25,10.89,89.11,66.56,24.87,1.98,0.0,183
58237,Wyoming,WY,56,2021-06-01,10.89,89.11,68.06,24.28,1.98,0.0,183
58238,Wyoming,WY,56,2021-06-08,9.76,90.24,72.28,27.12,6.32,0.0,196


In [27]:
dataframe['LEVEL'] = dataframe['DSCI'] // 100
dataframe

,State,Postal code,Fips,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI,LEVEL
0,Alabama,AL,1,2000-01-04,0.00,100.00,47.98,16.35,0.00,0.0,164,1
1,Alabama,AL,1,2000-01-11,0.00,100.00,76.88,4.24,0.00,0.0,181,1
2,Alabama,AL,1,2000-01-18,0.00,100.00,75.71,4.24,0.00,0.0,180,1
3,Alabama,AL,1,2000-01-25,0.00,100.00,75.67,4.79,0.00,0.0,180,1
4,Alabama,AL,1,2000-02-01,0.00,100.00,50.07,0.17,0.00,0.0,150,1
...,...,...,...,...,...,...,...,...,...,...,...,...
58235,Wyoming,WY,56,2021-05-18,10.89,89.11,70.75,27.15,2.01,0.0,189,1
58236,Wyoming,WY,56,2021-05-25,10.89,89.11,66.56,24.87,1.98,0.0,183,1
58237,Wyoming,WY,56,2021-06-01,10.89,89.11,68.06,24.28,1.98,0.0,183,1
58238,Wyoming,WY,56,2021-06-08,9.76,90.24,72.28,27.12,6.32,0.0,196,1


In [28]:
dataframe.to_csv(base_url + r'files/drought.csv')